In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.9/802.9 kB 17.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
# YOLOv8 Training Script with Directory Creation and Tabular Metrics

import os
import yaml
from ultralytics import YOLO
import random
from PIL import Image
from tabulate import tabulate

# Set the base path for the project
BASE_PATH = '/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya'

# Define the path for the YAML configuration file
yaml_path = "/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Dataset/SingLDataset.v1i.yolov8/data.yaml"

# Create necessary directories
os.makedirs(os.path.join(BASE_PATH, 'Final_models_19_July'), exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, 'Final_runs_19_July'), exist_ok=True)

def validate_yaml_structure(data):
    required_keys = ['train', 'val', 'test', 'nc', 'names']
    for key in required_keys:
        if key not in data:
            raise KeyError(f"Missing required key '{key}' in data.yaml")

    if len(data['names']) != data['nc']:
        raise ValueError(f"Number of classes ({data['nc']}) does not match the length of 'names' list ({len(data['names'])})")

def validate_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
    except Exception as e:
        print(f"Warning: Invalid or corrupted image file: {image_path}")

def validate_dataset(yaml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)

    validate_yaml_structure(data)

    for split in ['train', 'val', 'test']:
        split_path = os.path.join(os.path.dirname(yaml_path), data[split])
        if not os.path.exists(split_path):
            raise FileNotFoundError(f"{split} dataset not found at {split_path}")

        image_files = [f for f in os.listdir(split_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not image_files:
            raise FileNotFoundError(f"No image files found in {split} dataset at {split_path}")

        print(f"Found {len(image_files)} images in {split} dataset")

        # Validate a random sample of images
        sample_size = min(10, len(image_files))
        for filename in random.sample(image_files, sample_size):
            image_file = os.path.join(split_path, filename)
            validate_image(image_file)

    print("Dataset validation passed successfully!")
    return data

def print_metrics(results, title):
    print(f"\n{title}:")
    table_data = []

    if hasattr(results, 'box'):
        table_data.extend([
            ["mAP50", f"{results.box.map50:.4f}"],
            ["mAP50-95", f"{results.box.map:.4f}"],
            ["Precision", f"{results.box.mp:.4f}"],
            ["Recall", f"{results.box.mr:.4f}"]
        ])

    if hasattr(results, 'speed'):
        table_data.extend([
            ["Preprocess Speed", f"{results.speed['preprocess']:.4f} ms"],
            ["Inference Speed", f"{results.speed['inference']:.4f} ms"],
            ["Postprocess Speed", f"{results.speed['postprocess']:.4f} ms"]
        ])

    if hasattr(results, 'results_dict'):
        for key, value in results.results_dict.items():
            table_data.append([key, f"{value:.4f}"])

    print(tabulate(table_data, headers=["Metric", "Value"], tablefmt="grid"))

    if hasattr(results, 'names'):
        print("\nClass Names:")
        class_names = [[i, name] for i, name in results.names.items()]
        print(tabulate(class_names, headers=["Class ID", "Name"], tablefmt="grid"))

# Validate the dataset and get the data
data = validate_dataset(yaml_path)

# Initialize the YOLOv8 model
model = YOLO('yolov8m.pt')  # Load a pretrained model for transfer learning

# Set up training arguments
args = {
    'data': yaml_path,
    'epochs': 25,
    'batch': 64,
    'device': 0,  # Use GPU. Set to 'cpu' if you don't have a GPU
    'project': os.path.join(BASE_PATH, 'Final_runs_19_July'),
    'name': 'Final_yolov8_indian_sign_language_19_July',
    'exist_ok': True,

}

# Train the model
results = model.train(**args)

# Print a summary of the training results
print_metrics(results, "Training Results")

# Evaluate the model on the validation set
val_results = model.val()

# Print validation results
print_metrics(val_results, "Validation Results")

# Perform inference on the test set
test_results = model.predict(source=os.path.join(os.path.dirname(yaml_path), data['test']), save=True, save_txt=True)

# Print test results
print("\nTest Results:")
print(f"Number of detections: {len(test_results)}")

# Save the trained model
model_save_path = os.path.join(BASE_PATH, 'Final_models_19_July', 'yolov8_indian_sign_language.pt')
try:
    model.save(model_save_path)
    print(f"\nModel saved successfully at: {model_save_path}")
except Exception as e:
    print(f"\nError saving model: {str(e)}")
    print(f"Please check if you have write permissions for the directory: {os.path.dirname(model_save_path)}")

print("\nTraining, evaluation, and model saving completed.")

Found 1212 images in train dataset
Found 341 images in val dataset
Found 195 images in test dataset
Dataset validation passed successfully!
Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/yolov8/data.yaml, epochs=25, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/runs, name=Final_yolov8_indian_sign_language, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=

train: Scanning /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/yolov8/train/labels.cache... 1212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1212/1212 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/yolov8/valid/labels.cache... 341 images, 0 backgrounds, 0 corrupt: 100%|██████████| 341/341 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/runs/Final_yolov8_indian_sign_language/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000256, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/runs/Final_yolov8_indian_sign_language
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


25 epochs completed in 0.096 hours.
Optimizer stripped from /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/runs/Final_yolov8_indian_sign_language/weights/last.pt, 52.1MB
Optimizer stripped from /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/runs/Final_yolov8_indian_sign_language/weights/best.pt, 52.1MB

Validating /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/runs/Final_yolov8_indian_sign_language/weights/best.pt...
Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,860,025 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


                   all        341        348      0.977      0.961      0.986      0.862
                     1          9          9      0.893          1      0.995      0.797
                     2         16         16          1      0.721      0.794      0.639
                     3         11         11          1      0.983      0.995      0.881
                     4         10         10      0.971          1      0.995       0.87
                     5         10         10          1      0.959      0.995      0.978
                     6         10         10       0.97          1      0.995       0.93
                     7         11         11      0.981          1      0.995      0.817
                     8         10         12          1      0.861      0.995      0.961
                     9         11         11      0.989          1      0.995      0.903
                     A         11         11      0.991          1      0.995      0.764
                     

val: Scanning /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/yolov8/valid/labels.cache... 341 images, 0 backgrounds, 0 corrupt: 100%|██████████| 341/341 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.18s/it]


                   all        341        348      0.977      0.961      0.986      0.861
                     1          9          9      0.893          1      0.995      0.797
                     2         16         16          1       0.72      0.794      0.639
                     3         11         11          1      0.982      0.995      0.881
                     4         10         10      0.971          1      0.995       0.87
                     5         10         10          1      0.959      0.995      0.978
                     6         10         10       0.97          1      0.995       0.93
                     7         11         11      0.981          1      0.995      0.817
                     8         10         12          1      0.861      0.995      0.961
                     9         11         11      0.989          1      0.995      0.883
                     A         11         11      0.991          1      0.995      0.764
                     

##Final Version

In [ ]:
def validate_dataset(yaml_path):
    try:
        with open(yaml_path, 'r') as file:
            data = yaml.safe_load(file)
    except Exception as e:
        print(f"Error reading YAML file: {str(e)}")
        return

    try:
        validate_yaml_structure(data)
    except Exception as e:
        print(f"Error in YAML structure: {str(e)}")
        return

    base_path = os.path.dirname(yaml_path)

    for split in ['train', 'val', 'test']:
        try:
            # Assume the path in YAML already includes 'images'
            images_path = os.path.join(base_path, data[split])
            # Construct labels path by replacing 'images' with 'labels'
            labels_path = images_path.replace('images', 'labels')

            if not os.path.exists(images_path):
                raise FileNotFoundError(f"Images folder not found for {split} dataset at {images_path}")
            if not os.path.exists(labels_path):
                raise FileNotFoundError(f"Labels folder not found for {split} dataset at {labels_path}")

            image_files = [f for f in os.listdir(images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            if not image_files:
                raise FileNotFoundError(f"No image files found in {split} dataset at {images_path}")

            print(f"Found {len(image_files)} images in {split} dataset")

            # Check for corresponding label files
            missing_labels = []
            for image_file in image_files:
                label_file = os.path.splitext(image_file)[0] + '.txt'
                label_path = os.path.join(labels_path, label_file)
                if not os.path.exists(label_path):
                    missing_labels.append(image_file)

            if missing_labels:
                print(f"Warning: {len(missing_labels)} images in {split} dataset are missing corresponding label files:")
                for img in missing_labels[:10]:  # Print first 10 missing labels
                    print(f"  - {img}")
                if len(missing_labels) > 10:
                    print(f"  ... and {len(missing_labels) - 10} more.")
            else:
                print(f"All images in {split} dataset have corresponding label files.")

            # Validate a random sample of images
            sample_size = min(10, len(image_files))
            for filename in random.sample(image_files, sample_size):
                image_file = os.path.join(images_path, filename)
                validate_image(image_file)

        except Exception as e:
            print(f"Error validating {split} dataset: {str(e)}")
            print("Traceback:")
            traceback.print_exc()
            return

    print("Dataset validation completed successfully!")
    return data

# Use the function
yaml_path = "/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Dataset/SingLDataset.v1i.yolov8/data.yaml"
data = validate_dataset(yaml_path)

if data:
    print("Dataset structure is valid.")
else:
    print("Dataset validation failed. Please check the error messages above.")

Found 3738 images in train dataset
All images in train dataset have corresponding label files.
Found 345 images in val dataset
All images in val dataset have corresponding label files.
Found 157 images in test dataset
All images in test dataset have corresponding label files.
Dataset validation completed successfully!
Dataset structure is valid.


In [ ]:
import os
import yaml
from ultralytics import YOLO
import random
from PIL import Image
from tabulate import tabulate
import matplotlib.pyplot as plt

# Set the base path for the project
BASE_PATH = '/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya'

# Define the path for the YAML configuration file
yaml_path = "/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Dataset/SingLDataset/data.yaml"

# Create necessary directories
os.makedirs(os.path.join(BASE_PATH, 'Final_models_19_July'), exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, 'Final_runs_19_July'), exist_ok=True)

def validate_yaml_structure(data):
    required_keys = ['train', 'val', 'test', 'nc', 'names']
    for key in required_keys:
        if key not in data:
            raise KeyError(f"Missing required key '{key}' in data.yaml")

    if len(data['names']) != data['nc']:
        raise ValueError(f"Number of classes ({data['nc']}) does not match the length of 'names' list ({len(data['names'])})")

def validate_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
    except Exception as e:
        print(f"Warning: Invalid or corrupted image file: {image_path}")

def validate_dataset(yaml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)

    validate_yaml_structure(data)

    for split in ['train', 'val', 'test']:
        split_path = os.path.join(os.path.dirname(yaml_path), data[split])
        if not os.path.exists(split_path):
            raise FileNotFoundError(f"{split} dataset not found at {split_path}")

        # Assume the path in YAML already includes 'images'
        images_path = split_path
        # Construct labels path by replacing 'images' with 'labels'
        labels_path = split_path.replace('images', 'labels')

        if not os.path.exists(images_path):
            raise FileNotFoundError(f"Images folder not found for {split} dataset at {images_path}")
        if not os.path.exists(labels_path):
            raise FileNotFoundError(f"Labels folder not found for {split} dataset at {labels_path}")

        image_files = [f for f in os.listdir(images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not image_files:
            raise FileNotFoundError(f"No image files found in {split} dataset at {images_path}")

        print(f"Found {len(image_files)} images in {split} dataset")

        # Check for corresponding label files
        missing_labels = []
        for image_file in image_files:
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(labels_path, label_file)
            if not os.path.exists(label_path):
                missing_labels.append(image_file)

        if missing_labels:
            print(f"Warning: {len(missing_labels)} images in {split} dataset are missing corresponding label files:")
            for img in missing_labels[:10]:  # Print first 10 missing labels
                print(f"  - {img}")
            if len(missing_labels) > 10:
                print(f"  ... and {len(missing_labels) - 10} more.")
        else:
            print(f"All images in {split} dataset have corresponding label files.")

        # Validate a random sample of images
        sample_size = min(10, len(image_files))
        for filename in random.sample(image_files, sample_size):
            image_file = os.path.join(images_path, filename)
            validate_image(image_file)

    print("Dataset validation completed!")
    return data

def print_metrics(results, title):
    print(f"\n{title}:")
    table_data = []

    if hasattr(results, 'box'):
        table_data.extend([
            ["mAP50", f"{results.box.map50:.4f}"],
            ["mAP50-95", f"{results.box.map:.4f}"],
            ["Precision", f"{results.box.mp:.4f}"],
            ["Recall", f"{results.box.mr:.4f}"]
        ])

    if hasattr(results, 'speed'):
        table_data.extend([
            ["Preprocess Speed", f"{results.speed['preprocess']:.4f} ms"],
            ["Inference Speed", f"{results.speed['inference']:.4f} ms"],
            ["Postprocess Speed", f"{results.speed['postprocess']:.4f} ms"]
        ])

    if hasattr(results, 'results_dict'):
        for key, value in results.results_dict.items():
            table_data.append([key, f"{value:.4f}"])

    print(tabulate(table_data, headers=["Metric", "Value"], tablefmt="grid"))

    if hasattr(results, 'names'):
        print("\nClass Names:")
        class_names = [[i, name] for i, name in results.names.items()]
        print(tabulate(class_names, headers=["Class ID", "Name"], tablefmt="grid"))

def plot_results(results, save_path):
    plt.figure(figsize=(12, 8))
    plt.plot(results.results_dict['train/box_loss'], label='Train Box Loss')
    plt.plot(results.results_dict['val/box_loss'], label='Val Box Loss')
    plt.plot(results.results_dict['train/cls_loss'], label='Train Cls Loss')
    plt.plot(results.results_dict['val/cls_loss'], label='Val Cls Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.savefig(save_path)
    plt.close()

# Validate the dataset and get the data
data = validate_dataset(yaml_path)

# Initialize the YOLOv8 model
model = YOLO('yolov8m.pt')  # Load a pretrained model for transfer learning

# Set up training arguments
args = {
    'data': yaml_path,
    'epochs': 50,  # Increased number of epochs
    'batch': 32,  # Reduced batch size to potentially improve generalization
    'device': 0,
    'project': os.path.join(BASE_PATH, 'Final_runs_19_July'),
    'name': 'Final_yolov8_indian_sign_language_19_July',
    'exist_ok': True,
    'patience': 10,  # Early stopping patience
    'augment': True,  # Enable built-in augmentations
    'mosaic': 1.0,  # Mosaic augmentation
    'mixup': 0.2,  # Mixup augmentation
    'cos_lr': True,  # Use cosine learning rate scheduler
    'seed': 42,  # Set a fixed seed for reproducibility
    'verbose': False


}

# Train the model
results = model.train(**args)

# Print a summary of the training results
print_metrics(results, "Training Results")

# Plot and save training results
plot_results(results, os.path.join(BASE_PATH, 'Final_runs_19_July', 'training_results.png'))

# Evaluate the model on the validation set
val_results = model.val()

# Print validation results
print_metrics(val_results, "Validation Results")

# Perform inference on the test set
test_results = model.predict(source=os.path.join(os.path.dirname(yaml_path), data['test'], 'images'), save=True, save_txt=True)

# Print test results
print("\nTest Results:")
print(f"Number of detections: {len(test_results)}")

# Save the trained model
model_save_path = os.path.join(BASE_PATH, 'Final_models_19_July', 'yolov8_indian_sign_language.pt')
try:
    model.save(model_save_path)
    print(f"\nModel saved successfully at: {model_save_path}")
except Exception as e:
    print(f"\nError saving model: {str(e)}")
    print(f"Please check if you have write permissions for the directory: {os.path.dirname(model_save_path)}")

print("\nTraining, evaluation, and model saving completed.")

Found 3738 images in train dataset
All images in train dataset have corresponding label files.
Found 345 images in val dataset
All images in val dataset have corresponding label files.
Found 157 images in test dataset
All images in test dataset have corresponding label files.
Dataset validation completed!
Ultralytics YOLOv8.2.60 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Dataset/SingLDataset/data.yaml, epochs=50, time=None, patience=10, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July, name=Final_yolov8_indian_sign_language_19_July, exist_ok=True, pretrained=True, optimizer=auto, verbose=False, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1

Exception ignored in: <function tqdm.__del__ at 0x7d66645b6440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1277, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'TQDM' object has no attribute 'last_print_t'


Model summary: 295 layers, 25,876,585 parameters, 25,876,569 gradients, 79.2 GFLOPs

Transferred 469/475 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July/Final_yolov8_indian_sign_language_19_July', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Dataset/SingLDataset/train/labels.cache... 3738 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3738/3738 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        345        348      0.971       0.95      0.985      0.856



50 epochs completed in 0.431 hours.
Optimizer stripped from /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July/Final_yolov8_indian_sign_language_19_July/weights/last.pt, 52.1MB
Optimizer stripped from /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July/Final_yolov8_indian_sign_language_19_July/weights/best.pt, 52.1MB

Validating /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July/Final_yolov8_indian_sign_language_19_July/weights/best.pt...
Ultralytics YOLOv8.2.60 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,860,025 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:04<00:00,  1.28it/s]


                   all        345        348      0.975      0.962      0.988      0.862
Speed: 0.1ms preprocess, 6.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July/Final_yolov8_indian_sign_language_19_July

Training Results:
+----------------------+-----------+
| Metric               | Value     |
+======================+===========+
| mAP50                | 0.9876    |
+----------------------+-----------+
| mAP50-95             | 0.8618    |
+----------------------+-----------+
| Precision            | 0.9748    |
+----------------------+-----------+
| Recall               | 0.9621    |
+----------------------+-----------+
| Preprocess Speed     | 0.1285 ms |
+----------------------+-----------+
| Inference Speed      | 6.7206 ms |
+----------------------+-----------+
| Postprocess Speed    | 0.7800 ms |
+----------------------+-----------+
| metrics/precision(B) | 0.9748    |
+------

KeyError: 'train/box_loss'

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.9/802.9 kB 19.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.9/802.9 kB 15.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
import cv2
from pytube import YouTube
from ultralytics import YOLO
import tempfile
import os

def download_youtube_video(url, output_path):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    stream.download(output_path=output_path, filename='video.mp4')
    return os.path.join(output_path, 'video.mp4')

def process_video(model, video_path, output_path, conf_threshold=0.25):
    print(f"Processing video: {video_path}")
    print(f"Output path: {output_path}")

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video properties: Width={width}, Height={height}, FPS={fps}, Total Frames={total_frames}")

    # Create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    if not out.isOpened():
        print(f"Error: Could not create output video file {output_path}")
        return

    frame_count = 0
    while frame_count < total_frames:
        ret, frame = cap.read()
        if not ret:
            print(f"Reached end of video at frame {frame_count}")
            break

        # Perform inference
        results = model(frame, conf=conf_threshold)

        # Draw bounding boxes
        annotated_frame = results[0].plot()

        # Write the frame
        out.write(annotated_frame)

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")

    cap.release()
    out.release()
    print(f"Video processing completed. Total frames processed: {frame_count}")

# Load your trained model
model_path = '/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Final_runs_19_July/Final_yolov8_indian_sign_language_19_July/weights/best.pt'
print(f"Loading model from: {model_path}")
model = YOLO(model_path)
print("Model loaded successfully")

# Function to process either a local video or YouTube link
def process_video_input(input_path, output_path, is_youtube=False):
    if is_youtube:
        with tempfile.TemporaryDirectory() as temp_dir:
            video_path = download_youtube_video(input_path, temp_dir)
            process_video(model, video_path, output_path)
    else:
        process_video(model, input_path, output_path)

# Example usage:
# For a local video file:
local_video_path = '/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/Demo_ISL.mp4'
output_path = '/content/drive/MyDrive/Coventry_Final_Year_Projects/Mourya/ISL.mp4'

print(f"Processing local video: {local_video_path}")
print(f"Output will be saved to: {output_path}")

try:
    process_video_input(local_video_path, output_path, is_youtube=False)
    print("Video processing completed successfully")
except Exception as e:
    print(f"An error occurred: {str(e)}")

print("Script execution completed")

Streaming output truncated to the last 5000 lines.

0: 640x448 1 I, 10.4ms
Speed: 2.7ms preprocess, 10.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 I, 10.4ms
Speed: 2.6ms preprocess, 10.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 I, 9.8ms
Speed: 2.5ms preprocess, 9.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 (no detections), 10.3ms
Speed: 2.7ms preprocess, 10.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 (no detections), 10.3ms
Speed: 2.6ms preprocess, 10.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 (no detections), 9.9ms
Speed: 2.5ms preprocess, 9.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 (no detections), 10.7ms
Speed: 2.7ms preprocess, 10.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 (no detections), 9.7ms
Speed: 2.5ms preprocess